# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9b6de2fcd0>
├── 'a' --> tensor([[ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304]])
└── 'x' --> <FastTreeValue 0x7f9b6de894c0>
    └── 'c' --> tensor([[ 0.6741, -1.5215, -1.3430,  0.8374],
                        [-0.8407, -0.3306, -0.4151,  0.4416],
                        [ 0.9422,  0.5976,  0.4450, -1.7133]])

In [4]:
t.a

tensor([[ 1.7557,  0.2245, -2.3674],
        [-1.0940,  0.7099,  0.5304]])

In [5]:
%timeit t.a

64.8 ns ± 0.0254 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9b6de2fcd0>
├── 'a' --> tensor([[-0.7584, -0.0110,  1.1087],
│                   [-1.6367, -0.0956,  0.3471]])
└── 'x' --> <FastTreeValue 0x7f9b6de894c0>
    └── 'c' --> tensor([[ 0.6741, -1.5215, -1.3430,  0.8374],
                        [-0.8407, -0.3306, -0.4151,  0.4416],
                        [ 0.9422,  0.5976,  0.4450, -1.7133]])

In [7]:
%timeit t.a = new_value

68.8 ns ± 0.0111 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304]]),
    x: Batch(
           c: tensor([[ 0.6741, -1.5215, -1.3430,  0.8374],
                      [-0.8407, -0.3306, -0.4151,  0.4416],
                      [ 0.9422,  0.5976,  0.4450, -1.7133]]),
       ),
)

In [10]:
b.a

tensor([[ 1.7557,  0.2245, -2.3674],
        [-1.0940,  0.7099,  0.5304]])

In [11]:
%timeit b.a

55.2 ns ± 0.0411 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4070, -0.7954, -1.0841],
               [-0.2359,  1.5264, -1.9859]]),
    x: Batch(
           c: tensor([[ 0.6741, -1.5215, -1.3430,  0.8374],
                      [-0.8407, -0.3306, -0.4151,  0.4416],
                      [ 0.9422,  0.5976,  0.4450, -1.7133]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.22 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

857 ns ± 3.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 672 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9abea7dd00>
├── 'a' --> tensor([[[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]],
│           
│                   [[ 1.7557,  0.2245, -2.3674],
│                    [-1.0940,  0.7099,  0.5304]]])
└── 'x' --> <FastTreeValue 0x7f9b4052e790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 75 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9b68dab0a0>
├── 'a' --> tensor([[ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304],
│                   [ 1.7557,  0.2245, -2.3674],
│                   [-1.0940,  0.7099,  0.5304]])
└── 'x' --> <FastTreeValue 0x7f9b68dabbe0>
    └── 'c' --> tensor([[ 0.6741, -1.5215, -1.3430,  0.8374],
                        [-0.8407, -0.3306, -0.4151,  0.4416],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 48.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 40.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]],
       
               [[ 1.7557,  0.2245, -2.3674],
                [-1.0940,  0.7099,  0.5304]]]),
    x: Batch(
           c: tensor([[[ 0.6741, -1.5215, -1.3430,  0.8374],
                       [-0.8407, -0.3306, -0.4151,  0.4416],
                       [ 0.9422,  0.5976,  0.4450, -1.7133]],
         

In [26]:
%timeit Batch.stack(batches)

324 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304],
               [ 1.7557,  0.2245, -2.3674],
               [-1.0940,  0.7099,  0.5304]]),
    x: Batch(
           c: tensor([[ 0.6741, -1.5215, -1.3430,  0.8374],
                      [-0.8407, -0.3306, -0.4151,  0.4416],
                      [ 0.9422,  0.5976,  0.4450, -1.7133],
                      [ 0.6741, -1.5215, -1.3430,  0.8374],
                      [-0.8407, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 332 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 855 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
